### Import packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install av
import torch, torchvision
import torchvision.transforms as transforms
import cv2
import os
from os import path
import json

### Extract frames from videos and place them inside video name folder

In [ ]:
def extract_frames(file_path, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    vidcap = cv2.VideoCapture(file_path)
    success, image = vidcap.read()
    count = 0
 
    while success:
      frame_path = os.path.join(target_dir, f'{count}.jpg')
      cv2.imwrite(frame_path, image)     # save frame as JPEG file
      count += 1
      success, image = vidcap.read()
      success, image = vidcap.read()
      success, image = vidcap.read()


root = '/content/drive/MyDrive/mini_dataset'
frame_root = '/content/drive/MyDrive/teste/hmdb51_frames'
labels_path = '/content/drive/MyDrive/teste/labels.csv'
class_name_to_label_path = '/content/drive/MyDrive/teste/class_name_to_label.json'

# read files
files = []

for class_name in os.listdir(root):
    for video_name in os.listdir(os.path.join(root, class_name)):
        files.append([os.path.join(class_name, video_name), class_name])

# normalize labels
class_name_to_label = {}
current_label = -1

for vid in files:
    label = class_name_to_label.get(vid[1], -1)

    if label == -1:
        current_label += 1
        class_name_to_label[vid[1]] = current_label
        label = current_label

    vid[1] = label


# save file paths
if not os.path.exists(os.path.split(labels_path)[0]):
    os.makedirs(os.path.split(labels_path)[0])

f = open(labels_path, 'w')

f.write('path,label\n')

for vid in files:
    f.write(f'{vid[0]},{vid[1]}\n')

f.close()

# save label normalization
if not os.path.exists(os.path.split(class_name_to_label_path)[0]):
    os.makedirs(os.path.split(class_name_to_label_path)[0])

with open(class_name_to_label_path, 'w') as json_file:
    json.dump(class_name_to_label, json_file, indent=4)

# extract frames
for i, vid in enumerate(files):
    file_path = os.path.join(root, vid[0])
    target_dir = os.path.join(frame_root, vid[0])

    extract_frames(file_path, target_dir)

    print(f'{i+1}/{len(files)}')


### Split labels csv file into train and validate - 80/20

In [7]:
import pandas as pd

def split(label):
    path = '/content/drive/MyDrive/teste'
    train = label.sample(frac=0.8, random_state=201)
    val = label.drop(train.index)
    train.to_csv(path + '/train.csv', mode='a', header=False)
    val.to_csv(path + '/val.csv', mode='a', header=False)


root = '/content/drive/MyDrive/teste'
frame_root = '/content/drive/MyDrive/teste/hmdb51_frames'
labels_path = '/content/drive/MyDrive/teste/labels.csv'
class_name_to_label_path = '/content/drive/MyDrive/teste/class_name_to_label.json'

labels_path = '/content/drive/MyDrive/teste/labels.csv'
labels_data = pd.read_csv(labels_path)
last_label = labels_data.tail(1)['label']
last_label = last_label.iloc[0]
max_labels = labels_data.nunique()
max_labels= max_labels['label']
index = 0

for i in range(max_labels):
    label = labels_data[labels_data['label'] == index]
    split(label)
    index +=1 


### Load the images to a model

In [24]:
import pandas as pd
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import matplotlib.pyplot as plt

frames_csv_path = '/content/drive/MyDrive/teste/labels.csv'
root_path = '/content/drive/MyDrive/teste/hmdb51_frames'

class MyDataset(Dataset):
  def __init__(self, frames_csv_file, root_dir, transform=None):
    self.frames_csv = pd.read_csv(frames_csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
      return len(self.frames_csv)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()

      img_name = os.path.join(self.root_dir,
                              self.frames_csv.iloc[idx, 0])
      print(img_name)
      image = Image.open(img_name).convert('RGB')
      tag = self.frames_csv.iloc[idx, 1:]
      sample = {'image': image, 'tag': tag}

      if self.transform:
          sample = self.transform(sample)

      return sample

frames_dataset = MyDataset(frames_csv_path, root_path)

fig = plt.figure()

for i in range(len(frames_dataset)):
    sample = frames_dataset[i]

    print(i, sample['image'].shape, sample['tag'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
   

    if i == 3:
        plt.show()
        break


/content/drive/MyDrive/teste/hmdb51_frames/catch/Seldin_Lipovic_-_Willi_Weber__Torwarttraining__catch_f_cm_np1_le_bad_4.avi


IsADirectoryError: ignored

<Figure size 432x288 with 0 Axes>